# YOLO

In [1]:
!pip install ultralytics
!pip install onnxruntime-gpu

In [2]:
from ultralytics import YOLO

In [3]:
model = YOLO("yolov8l.pt")
results = model("horse1.jpg")

for result in results:
  result.save(filename="result.jpg")


image 1/1 /content/horse1.jpg: 384x640 1 horse, 1846.3ms
Speed: 3.5ms preprocess, 1846.3ms inference, 1.8ms postprocess per image at shape (1, 3, 384, 640)


In [4]:
model.export(format="onnx")

Ultralytics YOLOv8.2.88 🚀 Python-3.10.12 torch-2.4.0+cu121 CPU (Intel Xeon 2.20GHz)

PyTorch: starting from 'yolov8l.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 84, 8400) (83.7 MB)

ONNX: starting export with onnx 1.16.2 opset 19...
ONNX: export success ✅ 9.9s, saved as 'yolov8l.onnx' (166.8 MB)

Export complete (18.6s)
Results saved to /content
Predict:         yolo predict task=detect model=yolov8l.onnx imgsz=640  
Validate:        yolo val task=detect model=yolov8l.onnx imgsz=640 data=coco.yaml  
Visualize:       https://netron.app


'yolov8l.onnx'

In [5]:
onnx_model = YOLO("yolov8l.onnx")
result = onnx_model("horse1.jpg")

WARNING ⚠️ Unable to automatically guess model task, assuming 'task=detect'. Explicitly define task for your model, i.e. 'task=detect', 'segment', 'classify','pose' or 'obb'.
Loading yolov8l.onnx for ONNX Runtime inference...

image 1/1 /content/horse1.jpg: 640x640 1 horse, 2857.6ms
Speed: 3.7ms preprocess, 2857.6ms inference, 1.7ms postprocess per image at shape (1, 3, 640, 640)


In [7]:
import onnxruntime as ort

sess_options = ort.SessionOptions()
sess_options.intra_op_num_threads = 3
sess = ort.InferenceSession('yolov8l.onnx', sess_options)
onnx_model = YOLO("yolov8l.onnx")
result = onnx_model("horse1.jpg")

WARNING ⚠️ Unable to automatically guess model task, assuming 'task=detect'. Explicitly define task for your model, i.e. 'task=detect', 'segment', 'classify','pose' or 'obb'.
Loading yolov8l.onnx for ONNX Runtime inference...

image 1/1 /content/horse1.jpg: 640x640 1 horse, 2808.5ms
Speed: 4.4ms preprocess, 2808.5ms inference, 1.7ms postprocess per image at shape (1, 3, 640, 640)


In [9]:
import onnxruntime as ort

sess_options = ort.SessionOptions()
sess_options.intra_op_num_threads = 8
sess = ort.InferenceSession('yolov8l.onnx', sess_options)
onnx_model = YOLO("yolov8l.onnx")
result = onnx_model("horse1.jpg")

WARNING ⚠️ Unable to automatically guess model task, assuming 'task=detect'. Explicitly define task for your model, i.e. 'task=detect', 'segment', 'classify','pose' or 'obb'.
Loading yolov8l.onnx for ONNX Runtime inference...

image 1/1 /content/horse1.jpg: 640x640 1 horse, 2784.2ms
Speed: 3.7ms preprocess, 2784.2ms inference, 1.8ms postprocess per image at shape (1, 3, 640, 640)


In [10]:
import onnxruntime as ort

sess_options = ort.SessionOptions()
sess_options.intra_op_num_threads = 4
sess = ort.InferenceSession('yolov8l.onnx', sess_options)
onnx_model = YOLO("yolov8l.onnx")
result = onnx_model("horse1.jpg")

WARNING ⚠️ Unable to automatically guess model task, assuming 'task=detect'. Explicitly define task for your model, i.e. 'task=detect', 'segment', 'classify','pose' or 'obb'.
Loading yolov8l.onnx for ONNX Runtime inference...

image 1/1 /content/horse1.jpg: 640x640 1 horse, 2804.3ms
Speed: 28.8ms preprocess, 2804.3ms inference, 1.7ms postprocess per image at shape (1, 3, 640, 640)


# SKLEARN

In [11]:
!pip install skl2onnx

In [12]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
# from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier

iris = load_iris()
X, y = iris.data, iris.target
X = X.astype(np.float32)
X_train, X_test,y_train, y_test = train_test_split(X,y)

model = MLPClassifier()
model.fit(X_train, y_train)


MLPClassifier()

In [13]:
X[0]

array([        5.1,         3.5,         1.4,         0.2], dtype=float32)

In [14]:
import skl2onnx

onnx_model = skl2onnx.to_onnx(model, X[0])
with open("scikit_learn_onnx_model.onnx","wb")as f:
  f.write(onnx_model.SerializeToString())


In [15]:
!pip install onnxruntime

In [16]:
import onnxruntime as ort

onnx_model = ort.InferenceSession(
            "/content/scikit_learn_onnx_model.onnx",
             providers=["CPUExecutionProvider", "CUDAExecutionProvider"])



In [17]:
input_name = onnx_model.get_inputs()[0].name
output_name = onnx_model.get_outputs()[0].name
input_name, output_name

('X', 'output_label')

In [18]:
pred = onnx_model.run(
    [output_name],
    {input_name: X_test[0]}
    )
pred

[array([2], dtype=int64)]

# Pytorch

In [19]:
!pip install onnx
!pip install onnxscript

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 667.4/667.4 kB 9.1 MB/s eta 0:00:00


In [20]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class MyModel(nn.Module):

    def __init__(self):
        super(MyModel, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [21]:
torch_model = MyModel()
torch_input = torch.randn(1, 1, 32, 32)
onnx_model = torch.onnx.dynamo_export(torch_model, torch_input)

In [22]:
onnx_model.save("image_classification.onnx")

In [23]:
import onnx
onnx_program = onnx.load("image_classification.onnx")
onnx.checker.check_model(onnx_program)

In [24]:
import onnxruntime as ort

onnx_session = ort.InferenceSession(
    "/content/image_classification.onnx",
    providers=["CPUExecutionProvider"])

input_name = onnx_session.get_inputs()[0].name
output_name = onnx_session.get_outputs()[0].name

onnx_input = torch_input.detach().cpu().numpy()

input_name, output_name

pred = onnx_session.run(
    [output_name],
    {input_name: onnx_input})


pred

[array([[   -0.14049,    -0.10581,   -0.097569,   -0.034851,    -0.14039,    0.084876,   -0.093336,   -0.047788,    0.099776,    0.065922]], dtype=float32)]

In [25]:
onnxruntime_outputs  = pred

In [26]:
torch_outputs = torch_model(torch_input)
torch_outputs = onnx_model.adapt_torch_outputs_to_onnx(torch_outputs)

assert len(torch_outputs) == len(onnxruntime_outputs)
for torch_output, onnxruntime_output in zip(torch_outputs, onnxruntime_outputs):
    torch.testing.assert_close(torch_output, torch.tensor(onnxruntime_output))

print("PyTorch and ONNX Runtime output matched!")
print(f"Output length: {len(onnxruntime_outputs)}")
print(f"Sample output: {onnxruntime_outputs}")

PyTorch and ONNX Runtime output matched!
Output length: 1
Sample output: [array([[   -0.14049,    -0.10581,   -0.097569,   -0.034851,    -0.14039,    0.084876,   -0.093336,   -0.047788,    0.099776,    0.065922]], dtype=float32)]


## Tensorflow

In [27]:
!pip install tf2onnx

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 455.8/455.8 kB 7.0 MB/s eta 0:00:00


In [28]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications.resnet50 import ResNet50


model = ResNet50(weights="imagenet", include_top=True)

102967424/102967424 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step


In [29]:
import tf2onnx

input_data = tf.TensorSpec(model.inputs[0].shape,
                  dtype=model.inputs[0].dtype,
                  name=model.inputs[0].name)

onnx_model, _ = tf2onnx.convert.from_keras(
    model,
    [input_data])

In [30]:
onnx.save(onnx_model, "resnet50_model.onnx")

In [31]:
import numpy as np
from PIL import Image
import onnxruntime as ort
from tensorflow.keras.applications.resnet50 import decode_predictions


onnx_session = ort.InferenceSession(
    "resnet50_model.onnx",
    providers=["CPUExecutionProvider"]
)

input_name = onnx_session.get_inputs()[0].name
output_name = onnx_session.get_outputs()[0].name

image = Image.open("horse1.jpg")
image = image.resize((224, 224))
image = np.array(image).astype(np.float32)
image = np.expand_dims(image, axis=0)
onnx_input = image


pred = onnx_session.run(
    [output_name],
    {input_name: onnx_input}
)

pred


[array([[ 2.3856e-06,  1.7629e-05,  7.0427e-06,  9.9045e-05,  0.00036933,  0.00091795,  0.00021719,  3.6828e-05,   0.0001503,  2.6602e-05,  3.9028e-06,  1.9394e-06,  3.3991e-06,  5.4954e-06,  2.2527e-05,  1.4031e-05,  9.3577e-06,   0.0016598,  0.00010983,  9.7652e-06,  0.00011866,  1.9969e-05,  5.1492e-07,
          3.2335e-05,  1.6418e-05,  1.6417e-05,  1.0442e-05,  2.4405e-06,  2.0121e-05,  0.00014455,   2.232e-05,  0.00011881,  2.7457e-05,  4.8558e-05,  0.00019146,  6.2212e-06,  0.00031169,  1.3762e-06,  1.2081e-05,  3.5831e-05,  8.1303e-05,  1.5806e-05,  0.00010785,  0.00015422,  2.3961e-05,  4.5597e-06,
           2.082e-05,  3.1786e-06,  6.8761e-05,  0.00014127,  3.2177e-05,  0.00016783,  1.9627e-06,  1.0054e-06,  9.4069e-06,  3.7705e-06,  4.9248e-07,  6.2574e-06,  2.9554e-06,  1.1623e-05,  1.3723e-06,  5.0839e-06,  7.5869e-06,  1.2678e-06,  1.2981e-05,  1.6146e-05,  5.4423e-05,  1.8922e-05,  2.5072e-05,
          6.7003e-05,  9.5472e-07,   2.517e-06,  0.00010532,  1.0643e-05,  3

In [32]:
decode_predictions(pred[0])

35363/35363 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


[[('n02415577', 'bighorn', 0.60373545),
  ('n02412080', 'ram', 0.18106052),
  ('n02423022', 'gazelle', 0.039741453),
  ('n02114548', 'white_wolf', 0.031753436),
  ('n02437616', 'llama', 0.014114879)]]